In [26]:
import pandas as pd
import pandas_ml as pdml
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing


In [27]:
#Load data
df = pd.read_csv('data.csv');
#Replace NaN with 0
df = df.fillna(0);
df['DATE'] = pd.to_datetime(df['DATE']);
df.head()

,DATE,Average Wind Speed,PRCP,TMAX,TMIN,Fastest 2 minute wind speed,Fastest 5 second wind speed,Fog/Ice,Heavy/Freezing Fog,Thunder,Smoke/Haze,Power Outage
0,2017-12-31,4.25,0.00,15.0,-11.0,10.1,14.1,0.0,0.0,0.0,0.0,0.0
1,2017-12-30,8.72,0.00,15.0,-2.0,19.9,25.9,0.0,0.0,0.0,0.0,0.0
2,2017-12-29,7.16,0.00,17.0,8.0,16.1,19.9,0.0,0.0,0.0,0.0,0.0
3,2017-12-28,3.13,0.02,11.0,-20.0,8.1,11.0,1.0,0.0,0.0,0.0,0.0
4,2017-12-27,4.92,0.00,11.0,-17.0,17.0,21.0,0.0,0.0,0.0,0.0,0.0


In [28]:
#Add column for Delta T Max
temp = pd.Series(df.iloc[:,3]);
deltaTemp = pd.Series(df.iloc[:,3]);
deltaTemp[0] = 0;
for i in range (1, temp.shape[0]):
    deltaTemp[i] = temp[i] - temp[i-1];
df['Delta T Max'] = pd.Series(np.square(deltaTemp));

In [29]:
#Add column for Delta T min
temp = pd.Series(df.iloc[:,4]);
deltaTemp = pd.Series(df.iloc[:,4]);
deltaTemp[0] = 0;
for i in range (1, temp.shape[0]):
    deltaTemp[i] = temp[i] - temp[i-1];
df['Delta T Min'] = pd.Series(np.square(deltaTemp));

In [30]:
#Add columns for wind squared
df['Avg Wind Speed Squared'] = pd.Series(np.square(df['Average Wind Speed']), index=df.index)
df['5 second wind speed squared'] = pd.Series(np.square(df['Fastest 5 second wind speed']));
df['2 min wind speed squared'] = pd.Series(np.square(df['Fastest 2 minute wind speed']));
df = df[['Power Outage', 'PRCP','Delta T Min', 'Delta T Max', 'TMAX','TMIN','Fastest 5 second wind speed','5 second wind speed squared', 'Fastest 2 minute wind speed' ,'2 min wind speed squared','Average Wind Speed','Avg Wind Speed Squared','Fog/Ice', 'Heavy/Freezing Fog', 'Thunder','Smoke/Haze']];


In [31]:
#Normalize data
scaledData = preprocessing.MinMaxScaler().fit_transform(df.values);
finalData = pd.DataFrame(scaledData, index = df.index, columns = df.columns);

In [32]:
finalData.to_csv('Clustered Data', index = False);